## Baseline: random baseline

In [25]:
from datasets import load_dataset
import random
import numpy as np

seed =3
random.seed(seed)
np.random.seed(seed)


#### 
dataset_size = 5000
dataset_name ='all_train'
model_name="meta-llama/Meta-Llama-3.1-8B-Instruct"

json_dir = '/home/azureuser/cloudfiles/code/Users/jinlong.pang/LADR_LLM_alignment_data_refinement/open-instruct/data/train_data/'

all_train_dataset = load_dataset('json', data_files=json_dir+'all_train_data.jsonl')['train']



# random_indices = random.sample(len(all_train_dataset), dataset_size)
random_indices = np.random.permutation(len(all_train_dataset))[:dataset_size]

random_dataset = all_train_dataset.select(random_indices)

root_path = f"/home/azureuser/cloudfiles/code/Users/jinlong.pang/LADR_LLM_alignment_data_refinement/open-instruct/model_finetune_cluster/new_train_data/{model_name}/{dataset_name}/"

all_train_dataset.to_json(root_path + f"full_dataset.json")

random_dataset.to_json(root_path + f"random-{dataset_size//1000}k_dataset.json")


Creating json from Arrow format: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00, 46.72ba/s]


4339289

## Baseline: LESS (Influence-based)

In [ ]:
import torch 
from collections import Counter
import random
from datasets import load_dataset

seed =3
random.seed(seed)
np.random.seed(seed)

task='mmlu'
tot_dataset_name='all_train'
model_name="meta-llama/Meta-Llama-3.1-8B-Instruct"
dataset_size =40000


DATASET_LIST = ['flan_v2', 'oasst1', 'wizardlm', 'dolly', 'stanford_alpaca'] #all_train
# DATASET_LIST = ['flan_v2'] #all_train


task_list = ["mmlu"]
infl_score_all_tasks= []
for task in task_list:
    infl_score_all = []
    for dataset_name in DATASET_LIST:
        root_path = f"less-outputs/selected_data/{task}/"
        infl_scores = torch.load(root_path + f"{dataset_name}_influence_score.pt")
        if  dataset_name == 'dolly':
            infl_scores = infl_scores[:-40]
        print(f"dataset: {dataset_name} shape: {infl_scores.shape}")

        infl_score_all.append(infl_scores)

    infl_score_all = torch.cat(infl_score_all, dim=0)
    # infl_score_all_tasks.append(infl_score_all)

# infl_score_all_tasks = torch.cat(infl_score_all_tasks, dim=-1)
values, indices = torch.topk(infl_score_all, k=dataset_size)

############################################################################################

root_path = f"/home/azureuser/cloudfiles/code/Users/jinlong.pang/LADR_LLM_alignment_data_refinement/open-instruct/model_finetune_cluster/new_train_data/{model_name}/{tot_dataset_name}/"

data = load_dataset('json', data_files=root_path + 'full_dataset.json')
less_dataset = data['train'].select(indices)

less_dataset.to_json(root_path + f"less-{dataset_size//1000}k_dataset.json")

## Baseline: Label-filtered algorithm 

- tag: label-filtered

In [20]:
import torch 
from collections import Counter
import random
from datasets import load_dataset
import numpy as np

seed =3
random.seed(seed)
np.random.seed(seed)

dataset_name='all_train'
# model_name="meta-llama/Meta-Llama-3.1-8B-Instruct"
# model_name="gpt-4o-mini"
model_name= "mistralai/Mistral-7B-Instruct-v0.3"


dataset_size = 40000

all_train_dataset = load_dataset('json', data_files =f"/home/azureuser/cloudfiles/code/Users/jinlong.pang/LADR_LLM_alignment_data_refinement/open-instruct/data/train_data/{dataset_name}_data.jsonl")



label_path = f"/home/azureuser/cloudfiles/code/Users/jinlong.pang/LADR_LLM_alignment_data_refinement/open-instruct/model_finetune_cluster/new_train_data/{model_name}/{dataset_name}/output_labels_revised.pt"
labels = torch.load(label_path)


label_counts = Counter(labels)

print(f"Counter(labels): {label_counts}")


label_filtered_indices = []
for target_label in label_counts.keys():
    if len(label_filtered_indices) == dataset_size:
        break   

    indices = [i for i, label in enumerate(labels) if label == target_label]

    if dataset_size - len(label_filtered_indices) > len(indices):
        label_filtered_indices.extend(indices)
    else:
        random_indices = np.random.permutation(len(indices))[:dataset_size-len(label_filtered_indices)]
        label_filtered_indices.extend(random_indices)

print(f"data size: {len(label_filtered_indices)}")

label_filtered_dataset = all_train_dataset['train'].select(label_filtered_indices)

label_filtered_labels = np.array(labels)[label_filtered_indices].tolist()

root_path = f"/home/azureuser/cloudfiles/code/Users/jinlong.pang/LADR_LLM_alignment_data_refinement/open-instruct/model_finetune_cluster/new_train_data/{model_name}/{dataset_name}/"

label_filtered_dataset.to_json(root_path + f"label-filtered-{dataset_size//1000}k_dataset.json")


Counter(labels): Counter({3: 118313, 4: 66181, 2: 60607, 1: 52908, 0: 2436, 5: 487})
data size: 40000


Creating json from Arrow format: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 40/40 [00:00<00:00, 42.76ba/s]


66928993

## Baseline: diversity-filtered

- tag: diversity-filtered

In [46]:
from transformers import AutoTokenizer, AutoModel
import torch
import torch.nn.functional as F
from datasets import load_dataset, Dataset
import random
import numpy as np
from tqdm import tqdm
import os

seed = 3
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

def cosDistance(sample_embedding, selected_embeddings, k_near=10):
    # 确保在GPU上执行
    sample_embedding = sample_embedding.to(device)
    selected_embeddings = selected_embeddings.to(device)

    similarity_vector = torch.matmul(selected_embeddings, sample_embedding)
    distance_vector = 1.0 - similarity_vector

    if selected_embeddings.size(0) > k_near:
        distance_vector, _ = torch.topk(distance_vector, k=k_near, dim=0)
    
    mean_distance = distance_vector.mean().item()  # .item()确保从张量转换为python标量
    return mean_distance

def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0]  # First element of model_output contains all token embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)

def process_dialog(dialog):
    conversation = ""
    for message in dialog['messages']:
        conversation += f"### {message['role']}: {message['content']}\n"
    return {"features": conversation}

def embed_text(batch):
    encoded_inputs = tokenizer(batch['features'], padding=True, truncation=True, return_tensors="pt").to(device)
    with torch.no_grad():
        model_outputs = model(**encoded_inputs)
    sentence_embeddings = mean_pooling(model_outputs, encoded_inputs['attention_mask'])
    embeddings = F.normalize(sentence_embeddings, p=2, dim=1)
    
    batch['embeddings'] = embeddings.cpu().numpy().tolist()  # 确保从GPU传递回CPU进行保存
    return batch

##############################################################################################################################

dataset_name = 'all_train'
# model_name = "meta-llama/Meta-Llama-3.1-8B-Instruct"
# model_name= "mistralai/Mistral-7B-Instruct-v0.3"
model_name="gpt-4o-mini"

dataset_size = 40000

threshold = 0.5
k_near = 10

if not os.path.exists(f"{dataset_name}_embeddings.parquet"):
    data = load_dataset('json', data_files=f"/home/azureuser/cloudfiles/code/Users/jinlong.pang/LADR_LLM_alignment_data_refinement/open-instruct/data/train_data/{dataset_name}_data.jsonl")
    data['train'] = data['train'].map(process_dialog, batched=False)

    embedding_model_name = "BAAI/bge-large-en-v1.5"
    tokenizer = AutoTokenizer.from_pretrained(embedding_model_name)
    model = AutoModel.from_pretrained(embedding_model_name).to(device)
    model = torch.nn.DataParallel(model)

    data['train'] = data['train'].map(embed_text, batched=True, batch_size=2048)
    data['train'].to_parquet(f'{dataset_name}_embeddings.parquet')
    print(f"Embeddings saved to {dataset_name}_embeddings.parquet")

#########################################################################################################################

root_path = f"/home/azureuser/cloudfiles/code/Users/jinlong.pang/LADR_LLM_alignment_data_refinement/open-instruct/model_finetune_cluster/new_train_data/{model_name}/{dataset_name}/"
labels = torch.load(root_path + "output_labels_revised.pt")

# 定义标签和优先级
label_priority = [5, 4, 3, 2]
label_indices = {label: [idx for idx, lbl in enumerate(labels) if lbl == label] for label in label_priority}

embedding_dataset = load_dataset('parquet', data_files=f'{dataset_name}_embeddings.parquet')['train']

# 选择高质量样本
selected_embeddings = None
selected_samples = []

for label in label_priority:
    if len(selected_samples) >= dataset_size:
        break
    
    embedding_subset = embedding_dataset.select(label_indices[label])
    
    for sample in tqdm(embedding_subset, desc=f"Processing high-quality samples (label={label})"):
        sample_embedding = torch.tensor(sample['embeddings']).to(device)  # 确保 sample_embedding 在 GPU 上
        
        if selected_embeddings is None:
            selected_embeddings = sample_embedding.unsqueeze(0)
            selected_samples.append(sample)
            continue
        
        if cosDistance(sample_embedding, selected_embeddings, k_near=k_near) < threshold:
            selected_embeddings = torch.cat((selected_embeddings, sample_embedding.unsqueeze(0)), dim=0)
            selected_samples.append(sample)
        
        if len(selected_samples) >= dataset_size:
            break
    print(f"Selected data size from label {label}: {len(selected_samples)}")

selected_dataset = Dataset.from_dict({col: [s[col] for s in selected_samples] for col in selected_samples[0].keys()})
print(f"Selected data size: {len(selected_dataset)}")

selected_dataset.to_json(root_path + f'diversity-filtered-{dataset_size//1000}k_dataset.json', orient='records', lines=True)


Using device: cuda


Processing high-quality samples (label=5): 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3829/3829 [00:03<00:00, 1216.97it/s]


Selected data size from label 5: 311


Processing high-quality samples (label=4): 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 59969/59969 [00:48<00:00, 1223.97it/s]


Selected data size from label 4: 6107


...

Selected data size from label 3: 25099


Processing high-quality samples (label=2):  52%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▍                                                                                                                                                                                                                                                                                                                                                                                                                                                      | 44717/86132 [00:46<00:42, 970.89it/s]


Selected data size from label 2: 40000
Selected data size: 40000


Creating json from Arrow format: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 40/40 [00:07<00:00,  5.70ba/s]


713261521

## Filetered method: Our Data selection method

#### check the label curation info

## Filtered 5: label-filtered based: all 5 samples  + 4-rated samples select using sorted long-tail score (reverse=True)

In [61]:
import torch 
import random
import numpy as np
from datasets import load_dataset
from collections import Counter

seed =3
random.seed(seed)
np.random.seed(seed)

dataset_name ='all_train'
# model_name = "meta-llama/Meta-Llama-3.1-8B-Instruct"
# model_name= "mistralai/Mistral-7B-Instruct-v0.3"
model_name="gpt-4o-mini"

dataset_size = 40000

# label curation reports
report_path = f"/home/azureuser/cloudfiles/code/Users/jinlong.pang/LADR_LLM_alignment_data_refinement/labeling/results/{model_name}/{dataset_name}/{dataset_name}_report.pt"
reports = torch.load(report_path)

# Part 1 (label-wise): label curation
corrupted_samples = [x[0] for x in reports.detection['label_error']]

cured_samples = []
cured_sample_labels = []
for sample in reports.curation['label_curation']:  # (idx, label, confidence)
    if sample[2] >= 1:  # confidence prob
        cured_samples.append(sample[0])
        cured_sample_labels.append((sample[0], sample[1]))

print(f"Cured sample size: {len(cured_sample_labels)}")

# Filter out some cured samples from corrupted instances
cured_samples_set = set(cured_samples)
corrupted_samples_total = [x for x in corrupted_samples if x not in cured_samples_set]

print(f"Corrupted samples total: {len(corrupted_samples_total)}")

# Change the original labels to the suggested label
root_path = f"/home/azureuser/cloudfiles/code/Users/jinlong.pang/LADR_LLM_alignment_data_refinement/open-instruct/model_finetune_cluster/new_train_data/{model_name}/{dataset_name}/"
labels = torch.load(root_path + "output_labels_revised.pt")

print(f"Original Counter(labels): {Counter(labels)}")

for sample_label in cured_sample_labels:
    labels[sample_label[0]] = sample_label[1]

print(f"Label size: {len(labels)}")
print(f"Revised Counter(labels): {Counter(labels)}")

# Filter out the low-quality samples
label_wise_filter_out_samples = set(corrupted_samples_total)
print(f"Label-wise filter out samples: {len(label_wise_filter_out_samples)}")

# Part 2 (feature-wise): handle the rare example
rare_samples = reports.detection['rare_example'][:len(reports.detection['rare_example']) // 2]
# rare_samples_filtered = [[sample[0], sample[1]] for sample in rare_samples]
rare_samples_filtered = np.array(rare_samples)[:, :2]  # 使用NumPy进行操作

print(f"Size of the remaining samples with high quality: {len(rare_samples_filtered)}")

labels = np.array(labels)

# 缓存标签索引
label_indices_cache = {label: np.where(labels == label)[0] for label in [5, 4, 3, 2, 1]}
print(f"Finished caching labels indices...")

filtered_indices = []

for target_label in [5, 4, 3, 2, 1]:
    if len(filtered_indices) >= dataset_size:
        break

    label_indices = label_indices_cache[target_label]
    available_size = dataset_size - len(filtered_indices)

    if available_size > len(label_indices):
        filtered_indices.extend(label_indices.tolist())
    else:
        # 获取对应标签的样本并排序
        label_samples = rare_samples_filtered[np.isin(rare_samples_filtered[:, 0], label_indices)]
        if len(label_samples) > 0:  # 确保样本不为空
            sorted_samples = label_samples[label_samples[:, 1].argsort()[::-1]][:available_size]
            filtered_indices.extend(sorted_samples[:, 0].astype(int).tolist())

    print("Size of the filtered dataset:", len(filtered_indices))
    

# Filter out the corrupted samples and reconstruct the dataset
data = load_dataset('json', data_files=root_path + 'full_dataset.json')

# 根据 filtered_indices 选择样本
filtered_dialogs = data['train'].select(filtered_indices)

# 保存过滤后的数据集
filtered_dialogs.to_json(root_path + f"filtered-{dataset_size//1000}k_dataset.json")


Cured sample size: 0
Corrupted samples total: 119476
Original Counter(labels): Counter({3: 118313, 4: 66181, 2: 60607, 1: 52908, 0: 2436, 5: 487})
Label size: 300932
Revised Counter(labels): Counter({3: 118313, 4: 66181, 2: 60607, 1: 52908, 0: 2436, 5: 487})
Label-wise filter out samples: 119476
Size of the remaining samples with high quality: 300932
Finished caching labels indices...
Size of the filtered dataset: 487
Size of the filtered dataset: 40000


Creating json from Arrow format: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 40/40 [00:01<00:00, 29.15ba/s]


101209769

In [50]:
label_indices

array([     3,     30,     56, ..., 300921, 300927, 300929])

In [ ]:
import torch
import random
import numpy as np
from datasets import load_dataset
from collections import Counter

# 设置随机种子
seed = 3
random.seed(seed)
np.random.seed(seed)

# Part 2 (feature-wise): 处理稀有样本
rare_samples = reports.detection['rare_example'][:len(reports.detection['rare_example']) // 2]
rare_samples_filtered = np.array(rare_samples)[:, :2]  # 使用NumPy进行操作

print(f"Size of the remaining samples with high quality: {len(rare_samples_filtered)}")

# 假设 labels 是 Python 列表，将其转换为 NumPy 数组
labels = np.array(labels)

# 缓存标签索引
label_indices_cache = {label: np.where(labels == label)[0] for label in [5, 4, 3, 2, 1]}
print(f"Finished caching labels indices...")

filtered_indices = []

# 过滤样本并进行排序
for target_label in [5, 4, 3, 2, 1]:
    if len(filtered_indices) >= dataset_size:
        break

    label_indices = label_indices_cache[target_label]
    available_size = dataset_size - len(filtered_indices)

    if available_size > len(label_indices):
        filtered_indices.extend(label_indices.tolist())
    else:
        # 获取对应标签的样本并排序
        label_samples = rare_samples_filtered[np.isin(rare_samples_filtered[:, 0], label_indices)]
        if len(label_samples) > 0:  # 确保样本不为空
            sorted_samples = label_samples[label_samples[:, 1].argsort()[::-1]][:available_size]
            filtered_indices.extend(sorted_samples[:, 0].astype(int).tolist())

    print("Size of the filtered dataset:", len(filtered_indices))

# 加载数据集并过滤掉无效样本
data = load_dataset('json', data_files=root_path + 'full_dataset.json')

# 选择样本并保存
filtered_dialogs = data['train'].select(filtered_indices)
filtered_dialogs.to_json(root_path + f"filtered-cured-{dataset_size//1000}k_dataset.json")
